In [3]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.datasets import mnist

In [17]:
def get_mnist_model():
  inputs = keras.Input(shape= (28*28,))
  features = layers.Dense(512, activation = "relu")(inputs)
  features = layers.Dropout(0.5)(features)
  outputs = layers.Dense(10, activation = "softmax")(features)
  model = keras.Model(inputs, outputs)
  return model

In [18]:
(images, labels), (test_images, test_labels) = mnist.load_data()

In [19]:
images.shape

(60000, 28, 28)

In [20]:
images = images.reshape((60000, 28*28)).astype("float32")/255
test_images = test_images.reshape((10000, 28*28)).astype("float32")/255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

In [22]:
model = get_mnist_model()
model.compile(optimizer = "rmsprop", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])
model.fit(train_images, train_labels, epochs = 3, validation_data=(val_images, val_labels))

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8656 - loss: 0.4421 - val_accuracy: 0.9567 - val_loss: 0.1485
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9504 - loss: 0.1685 - val_accuracy: 0.9655 - val_loss: 0.1254
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9621 - loss: 0.1353 - val_accuracy: 0.9726 - val_loss: 0.1039


In [23]:
test_metrics = model.evaluate(val_images, val_labels)
print(test_metrics)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9738 - loss: 0.1038


In [25]:
test_metrics

[0.10392927378416061, 0.972599983215332]

In [24]:
predictions = model.predict(test_images)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [26]:
## Customize the worflow:
# Custom metrics

In [30]:
class RootMeanSquaredError(keras.metrics.Metric):
  # State variables in constructor
  def __init__(self, name = "rmsse", **kwargs):
    super().__init__(name = name, **kwargs)
    self.mse_sum = self.add_weight(name = "mse_sum", initializer = "zeros")
    self.total_samples = self.add_weight(name = "total_samples", initializer = "zeros", dtype = "int32")

  # logic is implemented in update_state
  def update_state(self, y_true, y_pred, sample_weight = None):
    y_true = tf.one_hot(y_true, depth = tf.shape(y_pred)[1])
    mse = tf.reduce_sum(tf.square(y_true - y_pred)) # sum of squared residuals
    self.mse_sum.assign_add(mse)
    num_samples = tf.shape(y_pred)[0]
    self.total_samples.assign_add(num_samples)

  # current value of metric
  def result(self):
    return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

  def reset_state(self):
    self.mse_sum.assign(0.)
    self.total_samples.assign(0)


In [31]:
model = get_mnist_model()
model.compile(optimizer = "rmsprop", loss = "sparse_categorical_crossentropy", metrics = ["accuracy", RootMeanSquaredError()])
model.fit(train_images, train_labels, epochs = 3, validation_data = (val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8634 - loss: 0.4537 - rmsse: 0.4401 - val_accuracy: 0.9591 - val_loss: 0.1423 - val_rmsse: 0.2511
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9531 - loss: 0.1659 - rmsse: 0.2702 - val_accuracy: 0.9682 - val_loss: 0.1088 - val_rmsse: 0.2163
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9628 - loss: 0.1322 - rmsse: 0.2391 - val_accuracy: 0.9732 - val_loss: 0.0997 - val_rmsse: 0.2032
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9697 - loss: 0.1108 - rmsse: 0.2157
